In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tsflex
from dataloaders import data_dict, data_set

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


args_dict = {
    "ucihar": {
        "root_path": r"../UCI HAR Dataset",
        "freq_save_path": r"../freq_data",
        "data_name": "ucihar"
    },
    "oppo": {
        "root_path": r"../OpportunityUCIDataset/dataset",
        "freq_save_path": r"../freq_data",
        "data_name": "oppo"
    },
    "pamap2": {
        "root_path": r"../PAMAP2_Dataset/Protocol/",
        "freq_save_path": r"../freq_data",
        "data_name": "pamap2"
    }
}


data_name = "pamap2"

args = dotdict(args_dict[data_name])    

args.difference = False 
args.sampling_freq =  100
args.train_vali_quote = 0.95
# for this dataset the window size is 128
args.windowsize = int(2.56 * args.sampling_freq)




args.drop_long = False

args.datanorm_type = "standardization" # None ,"standardization", "minmax"

args.wavename = "morl"
# if you want to use raw time series as input, set model_type as time
# if you want to use spectrogram as input, set model_type as freq
# if you want to use both as inputs, set model_type as cross
args.model_type = "time"

# if you want to do Leave one out cross validation, set exp_mode as "LOCV"
# if you want to do the given train test experiment, set exp_mode as "Given"
# if you want to do Semi non overlapping experiment, set exp_mode as "SOCV"
# if you want to do Full non overlapping experiment, set exp_mode as "FOCV"
args.exp_mode = "Given"

if args.exp_mode == "FOCV":
    args.displacement =  int(1 * args.windowsize) 
else:
    args.displacement =  int(0.5 * args.windowsize)     


args.batch_size = 32
args.shuffle = True
args.drop_last = False
dataset = data_dict[args.data_name](args)

About to load participants


In [7]:
sensors_0 = dataset.participants[0]
data_0 = [v for k,v in sensors_0.items()]
data_headers = [k for k,v in sensors_0.items()]

print(data_0[0].head)

<bound method NDFrame.head of                           time  acc_16_01_hand
0      1970-01-01 00:00:00.000        -9.91204
1      1970-01-01 00:00:00.010        -9.87531
2      1970-01-01 00:00:00.020        -9.72175
3      1970-01-01 00:00:00.030        -9.79920
4      1970-01-01 00:00:00.040        -9.49956
...                        ...             ...
408025 1970-01-01 01:08:00.250        -9.54108
408026 1970-01-01 01:08:00.260        -9.42932
408027 1970-01-01 01:08:00.270        -9.42745
408028 1970-01-01 01:08:00.280        -9.47246
408029 1970-01-01 01:08:00.290        -9.66621

[408030 rows x 2 columns]>


In [15]:
from tsflex.features import FeatureCollection, FeatureDescriptor, FuncWrapper
import numpy as np

fc = FeatureCollection(
    feature_descriptors=
    FeatureDescriptor(
        function=FuncWrapper(func=np.mean, output_names="np_means"),
        series_name="acc_16_01_hand",
        window="2.5s", stride="1.25s"
    )
)

fc.calculate(data=data_0[0].set_index("timestamp"))

Traceback (most recent call last):
  File "/Users/king/opt/anaconda3/lib/python3.9/site-packages/tsflex/features/feature_collection.py", line 630, in calculate
    calculated_feature_list = [self._executor(idx) for idx in idxs]
  File "/Users/king/opt/anaconda3/lib/python3.9/site-packages/tsflex/features/feature_collection.py", line 630, in <listcomp>
    calculated_feature_list = [self._executor(idx) for idx in idxs]
  File "/Users/king/opt/anaconda3/lib/python3.9/site-packages/tsflex/features/feature_collection.py", line 265, in _executor
    stroll, function = get_stroll_func(idx)
  File "/Users/king/opt/anaconda3/lib/python3.9/site-packages/tsflex/features/feature_collection.py", line 314, in get_stroll_function
    stroll = StridedRollingFactory.get_segmenter(**stroll_arg_dict)
  File "/Users/king/opt/anaconda3/lib/python3.9/site-packages/tsflex/features/segmenter/strided_rolling_factory.py", line 80, in get_segmenter
    raise ValueError("Cannot segment a sequence-series with a t

RuntimeError: Feature Extraction halted due to error while extracting one (or multiple) feature(s)! See stack trace above.